In [ ]:
pip install xgboost lightgbm

In [ ]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
best_config_xgb = {'colsample_bytree': 0.8, 'gamma': 0.4, 'learning_rate': 0.012, 'max_depth': 8, 'min_child_weight': 5, 'n_estimators': 250, 'reg_alpha': 0, 'reg_lambda': 1.0, 'scale_pos_weight': 3, 'subsample': 0.68}
best_config_lgbm = {'colsample_bytree': 0.7170644193332939, 'learning_rate': 0.03412358349737991, 'max_depth': 10, 'n_estimators': 153, 'num_leaves': 33, 'reg_alpha': 0.11878116785878495, 'subsample': 0.8781268957112172}
best_elastic_search = {'max_iter': 1000, 'l1_ratio': 0.7, 'fit_intercept': True, 'alpha': 0.1}
best_meta = {'l1_ratio': 0.7, 'alpha': 5.0}
# Define base models
base_models = [
    ('elastic_net', make_pipeline(StandardScaler(), ElasticNet(**best_elastic_search))),
    ('xgb', XGBRegressor(**best_config_xgb)),
    ('lgbm', LGBMRegressor(**best_config_lgbm))
]
# Initialize StackingRegressor
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=ElasticNet(**best_meta))

stacking_regressor.fit(X_train, y_train['Month 4'].values)

# Submission

In [64]:
y_pred = stacking_regressor.predict(X_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


In [65]:
submission = pd.read_csv("y_test_example.csv", sep = ";")
submission['Month 4'] = y_pred
submission['Month 4'] = submission['Month 4'].apply(lambda x: np.max([0, x]))
submission

,index,Month 4
0,490548,4169.451837
1,2167957,0.300267
2,1999013,2.131073
3,469221,354.745346
4,1171716,0.326570
...,...,...
440738,849490,1.034734
440739,1199858,0.394458
440740,1007643,0.315126
440741,1647728,0.326570


In [69]:
submission.to_csv("submission.csv", sep = ";", index = False)